### B.1.1.7

#### The selection of suitable loci (done on Tue 22. Dec. 2020)
To identify suitable targets for primer/probe design, we downloaded 1,136 sequences from the GISAID repository filtered during a collection time spanning 1 - 21 December 2020. We focused on the spike gene because lineage B.1.1.7 contains a number of spike gene mutations, including two deletions (ΔH69/ΔV70 and ΔY144) that we focused on for designing a specific assay. 

I cut the locus encoding the spike protein and used the *MAFFT* alignment tool (with the parameter - auto) to align all the sequences against the WUHAN reference (NCBI ID: NC_045512.2). 

In [9]:
%%bash

# "msa_1221.fasta" is a pre-filtered nucleotide MSA file downladed from the GISAID repository 22.12.2020
#  the WUHAN reference is always used as the first sequence in the GISAID MSA files
grep -m 1 ">" msa_1221.fasta | cut -d">" -f2 >  sars2_allSeqs_til21stDec2020_andRefWuhan.list
# to reduce computational time, I used only sequences collected in Dec 2020
# getting unique sequence IDs
grep -P "2020-12-" msa_1221.fasta | cut -d">" -f2 >> sars2_allSeqs_til21stDec2020_andRefWuhan.list
# star-end positions of the spike protein in the aligned WUHAN sequence: 22412-26369
# I called the spike locus of all sequences listed in "sars2_allSeqs_til21stDec2020_andRefWuhan.list"
count=$(wc -l sars2_allSeqs_til21stDec2020_andRefWuhan.list | cut -d" " -f1)
for ((i=1; i<$(($count+1)); i++))
do
ID=$(sed -n ''$i'p' sars2_allSeqs_til21stDec2020_andRefWuhan.list | cut -d" " -f1)
echo ">"$ID  >>  sars2_allSeqs_til21stDec2020_andRefWuhan_Spike.fa
grep -A 1 -m 1 $ID  msa_1221.fasta | grep -v ">" | cut -c22412-26369 | tr -d '-' | tr -d '\n' | tr -d ' '  >>  sars2_allSeqs_til21stDec2020_andRefWuhan_Spike.fa
echo "" >>  sars2_allSeqs_til21stDec2020_andRefWuhan_Spike.fa
done

# using 4 CPUS, I run the mafft tool with default settings
mafft --thread 4 --auto sars2_allSeqs_til21stDec2020_andRefWuhan_Spike.fa > sars2_allSeqs_til21stDec2020_andRefWuhan_Spike_mafft.fa

#### Downstream analysis
Twelve sequences (1.06 %) contained ambiguous signal in the loci of deletions and were not used in the downstream analysis. We separated sequences into two groups: 1) those with the ΔH69/ΔV70 and ΔY144 deletions and 2) those without the deletions (Table 1). Using *SeaView*, we called 95 % consensus sequences for the ΔH69/ΔV70 and ΔY144 group and the No deletions group that were used to design primer and probe sets specific to either B.1.1.7 or all other SARS-CoV-2 variants, respectively. 

In [ ]:
%%bash
# quality checks of bases in the deleted loci (ΔH69/ΔV70 and ΔY144) 
# if a called base has ambiguous character, it is denotes as N
count=$(wc -l sars2_allSeqs_til21stDec2020_andRefWuhan.list | cut -d" " -f1)
for ((i=1; i<$(($count+1)); i++))
do
ID=$(sed -n ''$i'p' sars2_allSeqs_til21stDec2020_andRefWuhan.list | cut -d"|" -f2)
Del69_70=$(awk ' BEGIN {RS=">"}; /'$ID'\|/  { print ">"$0 } ' sars2_allSeqs_til21stDec2020_andRefWuhan_Spike_mafft.fa | grep -v ">" | tr -d '\n' | tr -d ' ' | cut -c203-208)
Del144=$(awk ' BEGIN {RS=">"}; /'$ID'\|/  { print ">"$0 } ' sars2_allSeqs_til21stDec2020_andRefWuhan_Spike_mafft.fa | grep -v ">" | tr -d '\n' | tr -d ' ' | cut -c428-430)
# using the output file, we can also compute the correlation of two deletions (ΔH69/ΔV70 and ΔY144) and to judge about their co-occurrence 
echo -e $ID"\t"$Del69_70"\t"$Del144 >> sars2_1stDec20202_21stDec20202_Spike_Qchecks.tsv
# The shorter deletion (ΔY144) always co-occurred with the longer deletion (ΔH69/ΔV70), whereas the (ΔH69/ΔV70) deletion occurs independently in 17 sequences (1.5 %). 
# Pearson's correlation coefficient of the deletions is 0.953. 

#### Quality checks of the selected primer/probe loci (done on Thu 4. Febr. 2021)
In a separate analysis to determine the prevalence of the ΔH69/ΔV70 and ΔY144 deletions in lineages other than B.1.1.7, we downloaded 416,778 spike protein sequences with the most recent data description file collected from the beginning of the pandemic through 29 January 2021. Using regular expressions (bash pattern matching command grep with the option -P for Perl-compatible regular expression), we searched for loci with both ΔH69/ΔV70 and ΔY144 deletions, and for loci without these deletions. In the regular expression, we kept fixed a few amino acids downstream and upstream from the deletions to omit any miscalling of the searched pattern. 

In [ ]:
%%bash
# "spikeprot0129.fasta" is a pre-filtered amino-acid MSA file downladed from the GISAID repository 22.12.2020
grep -c ">" spikeprot0129.fasta
# detection of both deletions (no HV, no Y): d69d70 and d144; with the check for unique sequence IDs
grep -B1 -P "HAISGT.{66}FLGVYHK" spikeprot0129.fasta | grep ">" | cut -d"/" -f2 | sort | uniq -c | awk ' { print $1"\t"$2} ' | wc -l
# full pattern (HV and Y), no deletion; with the check for unique sequence IDs
grep -B1 -P "HAIHVSGT.{66}FLGVYYHK" spikeprot0129.fasta | grep ">" | cut -d"/" -f2 | sort | uniq -c | awk ' { print $1"\t"$2} ' | wc -l
# only d144 (only HV, no Y); with the check for unique sequence IDs
grep -B1 -P "HAIHVSGT.{66}FLGVYHK" spikeprot0129.fasta | grep ">" | cut -d"/" -f2 | sort | uniq -c | awk ' { print $1"\t"$2} ' | wc -l
# only d69d70 (only Y, no HV); with the check for unique sequence IDs
grep -B1 -P "HAISGT.{66}FLGVYYHK" spikeprot0129.fasta | grep ">" | cut -d"/" -f2 | sort | uniq -c | awk ' { print $1"\t"$2} ' | wc -l
# detection of both deletions (no HV, no Y): d69d70 and d144; with the time-dependent sorting
grep -B1 -P "HAISGT.{66}FLGVYHK" spikeprot0129.fasta | grep ">" | grep -oP "\|202[01]-..-" | sort | uniq -c
# only d69d70 (only Y, no HV); with the time-dependent sorting
grep -B1 -P "HAISGT.{66}FLGVYYHK" spikeprot0129.fasta | grep ">" | grep -oP "\|202[01]-..-" | sort | uniq -c

#### Results
Our analysis of the prevalence of both ΔH69/ΔV70 and ΔY144 deletions in lineages other than B.1.1.7, revealed a total of 29,872 sequences that possess both deletions, while 368,474 sequences do not have them. Based on the metadata file, we identified SARS-CoV-2 lineages across all called sequences with both deletions. Only five sequences (0.0167 %) out of 29,872 records are not labelled as B.1.1.7, highlighted the notion that these two deletions are highly specific for the B.1.1.7 variant and make ideal targets for primer/probe design.